In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def check_k_anonymity(df, quasi_identifiers, k):
    # Raggruppare il dataframe per i quasi-identificatori e ottenere la dimensione di ogni gruppo
    group_sizes = df.groupby(quasi_identifiers).size()
    num_partitions = len(group_sizes)
    total_rows = len(df)
    
    # Verificare se tutti i gruppi hanno almeno k elementi e al massimo k*2 elementi
    is_k_anonymous = ((group_sizes >= k) & (group_sizes <= k*2)).all()
    
    # Stampa del risultato della verifica di k-anonimity
    if is_k_anonymous:
        print(f"Il dataset rispetta la k-anonimity per k compreso tra {k} e {k*2}")
    else:
        print(f"Il dataset non rispetta la k-anonimity per k compreso tra {k} e {k*2}")
        print("\nLe seguenti partizioni non rispettano la k-anonimity:")
        print(group_sizes[(group_sizes < k) | (group_sizes > k*2)])
    
    return is_k_anonymous

# Carica il dataset
anonymized_df = pd.read_csv('anonymized.csv')
original_df = pd.read_csv('generation/database.csv')   

# Definisci i quasi-identificatori e il valore di k
quasi_identifiers = ['age', 'gender', 'city', 'education', 'profession']
k = 3  # Sostituisci con il valore di k desiderato

# Verifica la k-anonimity
check_k_anonymity(anonymized_df, quasi_identifiers, k)


Il dataset rispetta la k-anonimity per k compreso tra 3 e 6


True

In [8]:
numerical_qis = ['age']
categorical_qis = ['gender', 'city', 'education', 'profession']
all_qis = categorical_qis + numerical_qis

In [9]:
import pandas as pd
import numpy as np

# Funzione per calcolare la media di un range di età
def calculate_mean(column):
    try:
        # Rimuoviamo parentesi quadre, se presenti
        col_range = column.strip('[]')
        # Dividiamo il range in due numeri
        col_split = col_range.split('-')
        # Calcoliamo la media
        return np.mean([int(col_split[0]), int(col_split[1])])
    except (IndexError, ValueError):
        # Se c'è un errore nell'indicizzazione o nella conversione, restituisci NaN
        return np.nan


# Calcoliamo la media dei range di età in anonymized_df
anonymized_df['age_mean'] = anonymized_df['age'].apply(calculate_mean)

# Calcoliamo la media dell'età in original_df
original_age_mean = original_df['age'].mean()

# Stampa dei risultati
print("Media dei range di età in anonymized_df:")
print(anonymized_df[['age', 'age_mean']])

print("\nMedia dell'età in original_df:")
print(original_age_mean)


Media dei range di età in anonymized_df:
         age  age_mean
0    [70-75]      72.5
1    [70-75]      72.5
2    [70-75]      72.5
3    [26-72]      49.0
4    [26-72]      49.0
..       ...       ...
995  [25-30]      27.5
996  [25-30]      27.5
997  [19-85]      52.0
998  [19-85]      52.0
999  [19-85]      52.0

[1000 rows x 2 columns]

Media dell'età in original_df:
53.646


In [10]:
import pandas as pd
import numpy as np

# Funzione per rimuovere parentesi quadre
def remove_brackets(value):
    return value.strip('[]')

# Funzione per calcolare la moda degli attributi categorici
def calculate_mode(df, categorical_qis):
    # Controllo per determinare se il DataFrame ha parentesi quadre nei valori
    if '[' in df.iloc[0][categorical_qis[0]]:
        # Se ci sono parentesi quadre, applica la funzione remove_brackets
        for col in categorical_qis:
            df[col] = df[col].apply(remove_brackets)

    # Calcola la moda per le colonne categoriche
    modes = df[categorical_qis].mode().iloc[0]

    return modes

# Esempio di utilizzo con entrambi i DataFrame
if __name__ == "__main__":
    # Carica il DataFrame anonymized_df (sostituire con il caricamento effettivo del file)
    anonymized_df = pd.read_csv('anonymized.csv')

    # Carica il DataFrame original_df (sostituire con il caricamento effettivo del file)
    original_df = pd.read_csv('generation/database.csv')

    # Definisci i quasi identificatori categorici
    categorical_qis = ['gender', 'city', 'age', 'profession', 'education']

    # Calcola la moda degli attributi categorici per anonymized_df
    modes_anonymized = calculate_mode(anonymized_df, categorical_qis)
    print("Moda degli attributi categorici per anonymized_df:")
    print(modes_anonymized)

    # Calcola la moda degli attributi categorici per original_df
    modes_original = calculate_mode(original_df, categorical_qis)
    print("\nModa degli attributi categorici per original_df:")
    print(modes_original)


Moda degli attributi categorici per anonymized_df:
gender           ANY-GENDER
city                    USA
age                   35-86
profession          ANY-JOB
education     ANY-EDUCATION
Name: 0, dtype: object

Moda degli attributi categorici per original_df:
gender                    Female
city                     Houston
age                           48
profession    Classical Composer
education            High School
Name: 0, dtype: object


In [11]:
def statistical_analysis(original_df, anonymized_df, numerical_qis, categorical_qis):
    """ Esegue un'analisi statistica comparativa tra il dataset originale e quello anonimizzato """
   
    # Filtra i dataset per considerare solo le colonne di interesse
    original_df = original_df[all_qis]
    anonymized_df = anonymized_df[all_qis]

    # Analisi delle colonne numeriche
    print("Analisi delle colonne numeriche:")
    for col in numerical_qis:
        anonymized_df[col] = anonymized_df[col].apply(calculate_mean)
        
        mean_original = original_df[col].mean()
        mean_anonymized = anonymized_df[col].mean()
        std_original = original_df[col].std()
        std_anonymized = anonymized_df[col].std()

        print(f"Colonna: {col}")
        print(f"  Media originale: {mean_original}")
        print(f"  Media anonimizzata: {mean_anonymized}")
        print(f"  Deviazione standard originale: {std_original}")
        print(f"  Deviazione standard anonimizzata: {std_anonymized}")

    # Analisi delle colonne categoriche
    print("Analisi delle colonne categoriche:")
    modes_original = calculate_mode(original_df, categorical_qis)
    modes_anonymized = calculate_mode(anonymized_df, categorical_qis)
    
    for col in categorical_qis:
        print(f"Colonna: {col}")
        print(f"  Moda originale: {modes_original[col]}")
        print(f"  Moda anonimizzata: {modes_anonymized[col]}")


In [12]:
statistical_analysis(original_df, anonymized_df, numerical_qis, categorical_qis)


Analisi delle colonne numeriche:
Colonna: age
  Media originale: 53.646
  Media anonimizzata: 53.6125
  Deviazione standard originale: 20.907314504732543
  Deviazione standard anonimizzata: 11.765862878653621
Analisi delle colonne categoriche:
Colonna: gender
  Moda originale: Female
  Moda anonimizzata: ANY-GENDER
Colonna: city
  Moda originale: Houston
  Moda anonimizzata: USA
Colonna: age
  Moda originale: 48
  Moda anonimizzata: 52.0
Colonna: profession
  Moda originale: Classical Composer
  Moda anonimizzata: ANY-JOB
Colonna: education
  Moda originale: High School
  Moda anonimizzata: ANY-EDUCATION


C:\Users\aleba\AppData\Local\Temp\ipykernel_8548\1317917204.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anonymized_df[col] = anonymized_df[col].apply(calculate_mean)
